複数の動画を結合して、GitHub / SNS 向けの GIF・MP4 を作成します。
画面録画のデモ用途に合わせて、特定箇所のズームアップ機能もオプションで利用可能です。
対応入力形式: `.mov` `.mp4` `.avi` `.mkv` `.webm`


##### 出力形式の選び方

| 形式 | 特性 | 向いている用途 | 向かない用途 |
|---|---|---|---|
| GIF | 自動再生・ループ・あらゆる Markdown で動く。256色・ファイル大 | GitHub README（自動再生必須）・Zenn / Qiita | ファイルサイズが厳しい場面 |
| MP4 | 高品質・小容量・フルカラー。再生にはプレイヤーが必要 | X / Slack / GitHub README（クリック再生で可） | 自動再生・ループが必須の場面 |


##### 使い方 — 上から順に実行するだけ

| ステップ | 内容 | 操作 |
|:---:|---|---|
| ① | セットアップ | 自動 |
| ② | 動画をアップロード | ファイル選択ダイアログ |
| ③ | 結合順を指定 | ドロップダウン（1本の場合はスキップ） |
| ④ | 出力形式・品質・速度を設定 | ラジオボタン |
| ⑤ | プレビュー生成 | 自動 |
| ⑥ | プレビュー確認（MP4 で再生・秒数確認） | 自動 |
| ⑦ | ズーム設定（任意） | チェックボックス + イベント入力 |
| ⑧ | 最終出力 | 自動 |
| ⑨ | 保存先を選択して保存 | ラジオボタン |

> ※ アップロード上限（②）: 1ファイルあたり `〜2 GB`、複数ファイルは `2 GB × ファイル数` が目安


##### 参考: サービス別サイズ制約（2026/02 時点）

| プラットフォーム | 対応形式 | サイズ上限 |
|---|---|---|
| GitHub README | GIF / MP4 / MOV | 100 MB（動画）/ 10 MB（GIF） |
| Zenn | GIF | 3 MB |
| Qiita | GIF / MP4 | 100 MB |
| X（標準） | MP4 / MOV | 512 MB |
| Slack | GIF / MP4 / MOV 他 | 1 GB |

In [ ]:
# ① セットアップ
!apt-get install -y ffmpeg -q

from google.colab import files as colab_files, drive
import os, shutil, subprocess, ipywidgets as w
from IPython.display import display, Image, Video

print('✅ 準備完了')

In [ ]:
# ② アップロード（複数選択可）
print('📂 動画ファイルを選択してください（対応形式: .mov .mp4 .avi .mkv）')
uploaded = colab_files.upload()

VIDEO_EXT = ('.mov', '.mp4', '.avi', '.mkv', '.webm')
uploaded_videos = [f for f in uploaded.keys() if f.lower().endswith(VIDEO_EXT)]

if not uploaded_videos:
    print('❌ 動画ファイルが見つかりません')
else:
    print(f'\n✅ {len(uploaded_videos)} 本を検出しました:')
    for i, f in enumerate(uploaded_videos):
        size = os.path.getsize(f) / 1024 / 1024
        print(f'   [{i}] {f} ({size:.1f} MB)')

In [ ]:
# ③ 結合順（動画が1本の場合はスキップ）
if len(uploaded_videos) == 1:
    order_widgets = None
    print(f'✅ 動画が1本のため順番指定をスキップ: {uploaded_videos[0]}')
else:
    print('🔢 結合順を指定してください')
    order_widgets = [
        w.Dropdown(options=uploaded_videos, value=uploaded_videos[i],
                   description=f'{i+1}番目:', style={'description_width': '60px'},
                   layout=w.Layout(width='420px'))
        for i in range(len(uploaded_videos))
    ]
    display(w.VBox(order_widgets))

In [ ]:
# ④ 出力形式・品質・速度設定

# GIF プリセット（Cell 5・8 でも参照）
GIF_PRESETS = {
    'github': {'scale': 960,  'fps': 15, 'colors': 256, 'dither': 'floyd_steinberg'},
    'sns':    {'scale': 640,  'fps': 10, 'colors': 128, 'dither': 'bayer'},
    'hq':     {'scale': 1280, 'fps': 20, 'colors': 256, 'dither': 'floyd_steinberg'},
}

def make_toggle(widget, show_when):
    def _cb(change):
        widget.layout.display = '' if change['new'] == show_when else 'none'
    return _cb

format_sel = w.RadioButtons(
    options=[
        ('GIF  — ループ再生・GitHub README 向け', 'gif'),
        ('MP4 (H.264) — 高品質・小サイズ・SNS/Slack 向け', 'mp4'),
    ],
    value='gif', description='形式:', style={'description_width': '40px'},
    layout=w.Layout(width='420px')
)

# GIF 設定
gif_quality_sel = w.RadioButtons(
    options=[
        ('GitHub README用   960px / 15fps', 'github'),
        ('SNS・軽量用       640px / 10fps', 'sns'),
        ('高品質           1280px / 20fps', 'hq'),
        ('カスタム指定',                    'custom'),
    ],
    value='github', description='品質:', style={'description_width': '40px'}
)
gif_custom_scale  = w.BoundedIntText(value=960,  min=240, max=1920, description='横幅 px:',       layout=w.Layout(width='220px'), style={'description_width': '70px'})
gif_custom_fps    = w.BoundedIntText(value=15,   min=1,   max=30,   description='FPS:',           layout=w.Layout(width='180px'), style={'description_width': '70px'})
gif_custom_colors = w.BoundedIntText(value=256,  min=16,  max=256,  description='色数 (16-256):', layout=w.Layout(width='220px'), style={'description_width': '90px'})
gif_custom_dither = w.Dropdown(
    options=[('bayer（軽量）', 'bayer'), ('floyd_steinberg（高品質）', 'floyd_steinberg')],
    value='floyd_steinberg',
    description='ディザリング:', layout=w.Layout(width='300px'), style={'description_width': '90px'}
)
gif_custom_box = w.VBox(
    [gif_custom_scale, gif_custom_fps, gif_custom_colors, gif_custom_dither],
    layout=w.Layout(margin='0 0 0 20px', display='none')
)
gif_box = w.VBox([w.HTML('<b>🎨 GIF 品質</b>'), gif_quality_sel, gif_custom_box])
gif_quality_sel.observe(make_toggle(gif_custom_box, 'custom'), names='value')

# MP4 設定
mp4_quality_sel = w.RadioButtons(
    options=[
        ('高品質  (CRF 18) — ファイル大きめ', 18),
        ('標準    (CRF 23) — バランス重視',   23),
        ('軽量    (CRF 28) — ファイル小さめ', 28),
        ('カスタム指定',                      -1),
    ],
    value=23, description='品質:', style={'description_width': '40px'}
)
mp4_custom_crf = w.BoundedIntText(
    value=23, min=0, max=51, description='CRF (0-51):',
    layout=w.Layout(width='220px', display='none'), style={'description_width': '90px'}
)
mp4_scale = w.BoundedIntText(value=960, min=240, max=1920, description='横幅 px:', layout=w.Layout(width='220px'), style={'description_width': '70px'})
mp4_fps   = w.BoundedIntText(value=30,  min=1,   max=60,   description='FPS:',     layout=w.Layout(width='180px'), style={'description_width': '70px'})
mp4_audio = w.Checkbox(value=False, description='音声を保持する', layout=w.Layout(margin='4px 0'))
mp4_box = w.VBox([
    w.HTML('<b>🎨 MP4 品質</b>'), mp4_quality_sel, mp4_custom_crf,
    w.HTML('<b>📐 解像度・フレームレート</b>'), mp4_scale, mp4_fps,
    w.HTML('<b>🔊 音声</b>'), mp4_audio,
])
mp4_quality_sel.observe(make_toggle(mp4_custom_crf, -1), names='value')
settings_container = w.VBox([gif_box])

def on_format_change(change):
    settings_container.children = [gif_box] if change['new'] == 'gif' else [mp4_box]
format_sel.observe(on_format_change, names='value')

# 再生速度
speed_sel = w.RadioButtons(
    options=[('等倍 (1.0x)', 1.0), ('1.5倍速', 1.5), ('2.0倍速', 2.0), ('3.0倍速', 3.0), ('0.75倍速', 0.75), ('カスタム指定', -1)],
    value=1.0, description='速度:', style={'description_width': '40px'}
)
custom_speed = w.BoundedFloatText(
    value=1.0, min=0.1, max=10.0, step=0.1,
    description='倍率:', layout=w.Layout(width='200px', display='none'), style={'description_width': '40px'}
)
speed_sel.observe(make_toggle(custom_speed, -1), names='value')

display(w.VBox([
    w.HTML('<b>📦 出力形式</b>'), format_sel,
    w.HTML('<hr style="margin:12px 0">'),
    settings_container,
    w.HTML('<hr style="margin:12px 0">'),
    w.HTML('<b>⚡ 再生速度</b>'), speed_sel, custom_speed,
]))


In [ ]:
# ⑤ プレビュー生成（⑥ で秒数確認 → ⑦ でズーム設定 → ⑧ で最終出力）
# 設定変更後はこのセルから再実行

if order_widgets is None:
    ordered_videos = uploaded_videos
else:
    ordered_videos = [ow.value for ow in order_widgets]
    if len(set(ordered_videos)) != len(ordered_videos):
        dupes = [f for f in set(ordered_videos) if ordered_videos.count(f) > 1]
        raise ValueError(f'❌ 同じファイルが重複しています: {dupes}')

print(f'📋 結合順: {" → ".join(ordered_videos)}')

# 動画結合（複数の場合）
if len(ordered_videos) == 1:
    source = ordered_videos[0]
else:
    with open('list.txt', 'w') as f:
        for v in ordered_videos:
            f.write(f"file '{v}'\n")
    !ffmpeg -y -f concat -safe 0 -i list.txt -c copy combined.mov -loglevel warning
    source = 'combined.mov'
    print(f'✅ {len(ordered_videos)} 本を結合完了')

# ④ の設定からプレビュー用 fps・scale を決定（最終出力と同一解像度）
if format_sel.value == 'gif':
    if gif_quality_sel.value == 'custom':
        prev_scale, prev_fps = gif_custom_scale.value, gif_custom_fps.value
    else:
        p = GIF_PRESETS[gif_quality_sel.value]
        prev_scale, prev_fps = p['scale'], p['fps']
else:
    prev_scale, prev_fps = mp4_scale.value, mp4_fps.value

speed = custom_speed.value if speed_sel.value == -1 else speed_sel.value
print(f'   解像度: {prev_scale}px / fps: {prev_fps} / 速度: {speed}x')
print('⏳ プレビュー生成中（fast preset・音声なし）...')

!ffmpeg -y -i "{source}" -vf "fps={prev_fps},scale={prev_scale}:-2:flags=lanczos" -c:v libx264 -crf 23 -preset fast -an preview.mp4 -loglevel warning

size_mb = os.path.getsize('preview.mp4') / 1024 / 1024
print(f'\n✅ 完了 → preview.mp4 ({size_mb:.1f} MB)')
print('   ▶ ⑥ でプレビュー確認 → ⑦ でズーム設定（任意） → ⑧ で最終出力')


In [ ]:
# ⑥ プレビュー確認
# 再生・一時停止でズームしたい箇所のタイムスタンプを確認してください
display(Video('preview.mp4', embed=True, width=720))

In [ ]:
# ⑦ ズーム設定（任意）
# ⑥ のプレビューで確認したタイムスタンプ・エリアをグリッドから選択してください
#
# 時系列: 開始 → ズームイン → キープ → ズームアウト → 通常に戻る

AREA_GRID = {1:(0,0), 2:(1,0), 3:(2,0), 4:(0,1), 5:(1,1), 6:(2,1), 7:(0,2), 8:(1,2), 9:(2,2)}

zoom_enabled    = w.Checkbox(value=False, description='ズームを追加する', indent=False,
                              layout=w.Layout(margin='4px 0', width='auto'))
zoom_events_box = w.VBox([])
_zoom_getters   = []
_zoom_getters.clear()

def make_area_grid():
    """3x3 ToggleButton グリッドを作成。選択中エリアの番号(1-9)を返す get_area 関数も返す。"""
    LABELS = ['↖ 左上', '↑ 上中', '↗ 右上',
              '← 左中', '● 中央', '→ 右中',
              '↙ 左下', '↓ 下中', '↘ 右下']
    btns = [
        w.ToggleButton(
            description=LABELS[i], value=(i == 4),
            layout=w.Layout(width='88px', height='52px', margin='1px')
        ) for i in range(9)
    ]
    def on_click(change, clicked_idx):
        if change['new']:
            for j, b in enumerate(btns):
                if j != clicked_idx:
                    b.value = False
        elif not any(b.value for b in btns):
            btns[clicked_idx].value = True
    for idx, btn in enumerate(btns):
        btn.observe(lambda ch, i=idx: on_click(ch, i), names='value')
    grid = w.GridBox(btns, layout=w.Layout(
        grid_template_columns='repeat(3, 90px)', width='276px'
    ))
    get_area = lambda: next((i + 1 for i, b in enumerate(btns) if b.value), 5)
    return grid, get_area

def _lbl(text):
    """右揃えラベルを返す。"""
    return w.HTML(f'<div style="width:90px;text-align:right;padding-right:6px;line-height:32px">{text}</div>')

def _row(label, widget):
    return w.HBox([_lbl(label), widget])

def make_event_box(idx):
    grid, get_area = make_area_grid()
    max_z = w.BoundedFloatText(value=1.5, min=1.1, max=5.0, step=0.1,
        layout=w.Layout(width='120px'), tooltip='ズームのピーク倍率（1.1〜5.0）')
    start = w.BoundedFloatText(value=0.0, min=0, max=3600,
        layout=w.Layout(width='120px'), tooltip='ズームを開始する動画上の時刻')
    in_d  = w.BoundedFloatText(value=0.2, min=0, max=60,
        layout=w.Layout(width='120px'), tooltip='等倍 → 最大倍率へ拡大する時間')
    hold  = w.BoundedFloatText(value=0.5, min=0, max=60,
        layout=w.Layout(width='120px'), tooltip='最大倍率を維持する時間')
    out_d = w.BoundedFloatText(value=0.2, min=0, max=60,
        layout=w.Layout(width='120px'), tooltip='最大倍率 → 等倍へ縮小する時間')

    vbox = w.VBox([
        w.HTML(f'<b>── Zoomイベント #{idx} ──</b>'),
        w.HBox([
            w.VBox([w.HTML('<small>📍 ズームエリア</small>'), grid]),
            w.VBox([
                _row('最大倍率:', max_z),
                _row('開始(秒):', start),
                _row('イン(秒):', in_d),
                _row('キープ(秒):', hold),
                _row('アウト(秒):', out_d),
            ], layout=w.Layout(margin='0 0 0 16px')),
        ]),
    ], layout=w.Layout(border='1px solid #ddd', padding='8px', margin='4px 0'))

    def get_params():
        return {
            'area':     get_area(),
            'max_z':    max_z.value,
            'start':    start.value,
            'in_dur':   in_d.value,
            'hold_dur': hold.value,
            'out_dur':  out_d.value,
        }
    return vbox, get_area, get_params

add_btn = w.Button(description='＋ Zoomイベント追加', button_style='info',    layout=w.Layout(width='170px'))
del_btn = w.Button(description='－ 最後を削除',       button_style='warning', layout=w.Layout(width='150px'))

def _add_event(_):
    vbox, get_area, get_params = make_event_box(len(zoom_events_box.children) + 1)
    _zoom_getters.append(get_params)
    zoom_events_box.children = list(zoom_events_box.children) + [vbox]

def _del_event(_):
    if zoom_events_box.children:
        _zoom_getters.pop()
        zoom_events_box.children = list(zoom_events_box.children)[:-1]

add_btn.on_click(_add_event)
del_btn.on_click(_del_event)

zoom_panel = w.VBox([
    w.HTML('<br>'),
    zoom_events_box,
    w.HBox([add_btn, del_btn]),
    w.HTML(
        '<br><small>'
        '💡 操作手順: ⑥ のプレビュー動画を再生して秒数を確認 → 「ズームを追加する」にチェック '
        '→ Zoomイベントを追加して各項目を設定 → ⑧ を実行<br>'
        '💡 Zoomイベントが時間的に重複しないよう設定してください<br>'
        'ズームなしにするには「ズームを追加する」のチェックを外してください'
        '</small>'
    ),
], layout=w.Layout(display='none'))

def _on_zoom_toggle(change):
    zoom_panel.layout.display = '' if change['new'] else 'none'
zoom_enabled.observe(_on_zoom_toggle, names='value')

display(w.VBox([
    w.HTML('<b>🔍 ズーム設定</b>'),
    zoom_enabled,
    zoom_panel,
]))


In [ ]:
# ⑧ 最終出力
# ズーム設定変更後はこのセルだけ再実行

if not os.path.exists('preview.mp4'):
    raise RuntimeError('❌ preview.mp4 が見つかりません。⑤ を先に実行してください')

out_format  = format_sel.value
OUTPUT_NAME = f'demo.{out_format}'

def _ffmpeg(*args):
    """引数リストで ffmpeg を実行。quoting 問題を回避するため subprocess を使用。"""
    cmd = ['ffmpeg', '-y'] + [str(a) for a in args]
    proc = subprocess.run(cmd, capture_output=True, text=True)
    if proc.returncode != 0:
        print('❌ ffmpeg エラー:\n' + proc.stderr[-2000:])
    return proc.returncode

def _build_atempo(speed):
    """任意の速度に対応する atempo フィルタ文字列を構築（2.0 上限を連鎖で回避）。"""
    parts, s = [], speed
    while s > 2.0: parts.append('atempo=2.0'); s /= 2.0
    while s < 0.5: parts.append('atempo=0.5'); s *= 2.0
    parts.append(f'atempo={s:.4f}')
    return ','.join(parts)

def _build_zoompan(events, pw, ph, fps):
    """複数のズームイベントを ffmpeg zoompan フィルタ文字列に変換する。fps を明示的に指定することで速度変化を防ぐ。"""
    z_expr, x_expr, y_expr = '1', '0', '0'
    for ev in reversed(events):
        col, row = AREA_GRID[ev['area']]
        cx  = pw * (2 * col + 1) / 6
        cy  = ph * (2 * row + 1) / 6
        s, i, h, o, mz = ev['start'], ev['in_dur'], ev['hold_dur'], ev['out_dur'], ev['max_z']
        sf  = s * fps
        inf = (s + i) * fps
        hf  = (s + i + h) * fps
        of  = (s + i + h + o) * fps

        z_in  = f'(1+({mz}-1)*(on-{sf:.2f})/{max(inf-sf,1e-6):.2f})' if i > 0 else str(mz)
        z_out = f'({mz}-({mz}-1)*(on-{hf:.2f})/{max(of-hf,1e-6):.2f})' if o > 0 else '1'
        ev_z  = (f'if(lt(on,{inf:.2f}),{z_in},'
                 f'if(lt(on,{hf:.2f}),{mz},'
                 f'if(lt(on,{of:.2f}),{z_out},{z_expr})))')
        z_expr = f'if(gte(on,{sf:.2f}),{ev_z},{z_expr})'

        ev_x   = f'max(0,min(iw-iw/zoom,{cx:.2f}-iw/(2*zoom)))'
        ev_y   = f'max(0,min(ih-ih/zoom,{cy:.2f}-ih/(2*zoom)))'
        x_expr = f'if(between(on,{sf:.2f},{of:.2f}),{ev_x},{x_expr})'
        y_expr = f'if(between(on,{sf:.2f},{of:.2f}),{ev_y},{y_expr})'

    return f"zoompan=z='{z_expr}':x='{x_expr}':y='{y_expr}':d=1:s={pw}x{ph}:fps={fps}"

# ── ズームフィルタ準備 ────────────────────────────────────
zoom_filter = ''
if zoom_enabled.value and zoom_events_box.children:
    res = subprocess.run(
        ['ffprobe', '-v', 'error', '-select_streams', 'v:0',
         '-show_entries', 'stream=width,height,r_frame_rate', '-of', 'csv=p=0', 'preview.mp4'],
        capture_output=True, text=True
    )
    fields = res.stdout.strip().split(',')
    pw, ph = int(fields[0]), int(fields[1])
    num, den = map(int, fields[2].split('/'))
    detected_fps = num / den
    # 右側 VBox の構造: VBox[ HTML, HBox[ VBox[label,grid], VBox[max_z,start,in_d,hold,out_d,desc] ] ]
    events = [getter() for getter in _zoom_getters]
    zoom_filter = _build_zoompan(events, pw, ph, detected_fps)
    print(f'🔍 ズーム: {len(events)} イベント ({pw}x{ph} @ {detected_fps:.2f}fps)')

print(f'⏳ 最終変換中 ({out_format.upper()})...')

# ── GIF 出力 ──────────────────────────────────────────────
if out_format == 'gif':
    if gif_quality_sel.value == 'custom':
        fps, colors, dither = gif_custom_fps.value, gif_custom_colors.value, gif_custom_dither.value
    else:
        p = GIF_PRESETS[gif_quality_sel.value]
        fps, colors, dither = p['fps'], p['colors'], p['dither']
    # ズームがある場合は中間 MP4 にズーム処理を行い、その MP4 から GIF を生成する
    if zoom_filter:
        _ffmpeg('-i', 'preview.mp4', '-vf', zoom_filter,
                '-c:v', 'libx264', '-crf', 18, '-preset', 'fast', '-an',
                'zoomed.mp4', '-loglevel', 'warning')
        gif_source = 'zoomed.mp4'
    else:
        gif_source = 'preview.mp4'
    filter_str = (
        f"[0:v] fps={fps},split [a][b];"
        f"[a] palettegen=max_colors={colors}:stats_mode=full [p];"
        f"[b][p] paletteuse=dither={dither}:diff_mode=rectangle"
    )
    _ffmpeg('-i', gif_source, '-filter_complex', filter_str, OUTPUT_NAME, '-loglevel', 'warning')

# ── MP4 出力 ──────────────────────────────────────────────
elif out_format == 'mp4':
    crf   = mp4_custom_crf.value if mp4_quality_sel.value == -1 else mp4_quality_sel.value
    speed = custom_speed.value   if speed_sel.value       == -1 else speed_sel.value

    if zoom_filter:
        args = ['-i', 'preview.mp4']
        if mp4_audio.value:
            args += ['-i', source, '-map', '0:v', '-map', '1:a']
        args += ['-vf', zoom_filter, '-c:v', 'libx264',
                 '-crf', crf, '-preset', 'slow', '-pix_fmt', 'yuv420p']
        if mp4_audio.value:
            args += ['-filter:a', _build_atempo(speed), '-shortest']
        else:
            args += ['-an']
        args += [OUTPUT_NAME, '-loglevel', 'warning']
        _ffmpeg(*args)
    else:
        # ズームなし: preview.mp4 の映像をそのまま利用
        if mp4_audio.value:
            af_args = ['-filter:a', _build_atempo(speed)] if speed != 1.0 else []
            _ffmpeg('-i', 'preview.mp4', '-i', source,
                    '-map', '0:v', '-map', '1:a',
                    '-c:v', 'copy', *af_args, '-shortest',
                    OUTPUT_NAME, '-loglevel', 'warning')
        else:
            shutil.copy('preview.mp4', OUTPUT_NAME)

size_mb = os.path.getsize(OUTPUT_NAME) / 1024 / 1024
print(f'\n✅ 完了 → {OUTPUT_NAME} ({size_mb:.1f} MB)')

if out_format == 'gif' and size_mb > 15:
    print('\n⚠️  15MB 超えています。④ で以下を試してください:')
    print('   → 品質を「SNS・軽量用」に変更')
    print('   → または出力形式を MP4 に切り替えると大幅に削減できます')

print('\n▶ 最終プレビュー:')
if out_format == 'gif':
    display(Image(OUTPUT_NAME))
else:
    display(Video(OUTPUT_NAME, embed=True, width=720))


In [ ]:
# ⑨ 保存先を選択
save_sel = w.RadioButtons(
    options=[
        ('ローカルにダウンロード', 'local'),
        ('Google Drive に保存',   'drive'),
        ('両方',                  'both'),
    ],
    value='local', description='保存先:', style={'description_width': '60px'}
)
drive_path = w.Text(
    value=f'MyDrive/demo.{out_format}',
    description='Drive パス:',
    layout=w.Layout(width='380px', display='none'),
    style={'description_width': '80px'}
)

def on_save_change(change):
    drive_path.layout.display = '' if change['new'] in ('drive', 'both') else 'none'
save_sel.observe(on_save_change, names='value')

display(w.VBox([w.HTML('<b>💾 保存先</b>'), save_sel, drive_path]))

In [ ]:
# ⑨ 保存実行
method = save_sel.value

if method in ('local', 'both'):
    colab_files.download(OUTPUT_NAME)
    print('✅ ローカルにダウンロードしました')

if method in ('drive', 'both'):
    drive.mount('/content/drive')
    dest = f'/content/drive/{drive_path.value}'
    os.makedirs(os.path.dirname(dest), exist_ok=True)
    shutil.copy(OUTPUT_NAME, dest)
    print(f'✅ Drive に保存しました → {drive_path.value}')